# Modelling 

We train three models - XGBoost, Random Forest and Neural Network to predict which team would win based on the champion choosen and the role they are deployed in. 

In [18]:
import pandas as pd
import warnings
import os
import joblib
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
#from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score 
from sklearn.metrics import f1_score, precision_score, recall_score, fbeta_score

In [4]:
parent_dir = os.path.dirname(os.getcwd())
datasets_path = os.path.join(parent_dir, 'data', 'datasets')
transform_path = os.path.join(parent_dir, 'data', 'transform')
out_path = os.path.join(parent_dir, 'data', 'train')

## Pulling the Data 

In [5]:
# the data is taken from the transformation done in previous notebooks 
tf_df = pd.read_csv('tf_df.csv')
dataset = pd.read_csv('dataset.csv')#os.path.join(datasets_path, 

In [6]:
#check 
tf_df

,t1_top,t1_middle,t1_bottom,t1_utility,t1_jungle,t2_top,t2_middle,t2_bottom,t2_utility,t2_jungle,t1_top_wr,t1_middle_wr,t1_bottom_wr,t1_utility_wr,t1_jungle_wr,t2_top_wr,t2_middle_wr,t2_bottom_wr,t2_utility_wr,t2_jungle_wr,t1_top_role_wr,t1_middle_role_wr,t1_bottom_role_wr,t1_utility_role_wr,t1_jungle_role_wr,t2_top_role_wr,t2_middle_role_wr,t2_bottom_role_wr,t2_utility_role_wr,t2_jungle_role_wr,t1_top_kda,t1_middle_kda,t1_bottom_kda,t1_utility_kda,t1_jungle_kda,t2_top_kda,t2_middle_kda,t2_bottom_kda,t2_utility_kda,t2_jungle_kda,t1_top_role_kda,t1_middle_role_kda,t1_bottom_role_kda,t1_utility_role_kda,t1_jungle_role_kda,t2_top_role_kda,t2_middle_role_kda,t2_bottom_role_kda,t2_utility_role_kda,t2_jungle_role_kda,t1_win
0,Fiora,Corki,Samira,Pantheon,Kayn,Kassadin,Yone,Kaisa,Nautilus,Evelynn,52.233364,51.429580,49.760483,48.664122,50.689906,51.263264,48.790814,48.928255,49.915314,49.061958,52.334540,51.168224,49.843517,45.492289,50.805617,56.422018,48.712121,48.938667,50.100690,49.095238,1.827452,2.412114,2.293444,2.151817,2.611583,2.582077,1.901734,2.531856,2.354769,2.940007,1.827214,2.455387,2.301646,1.936946,2.627467,2.400353,1.981926,2.534069,2.355415,2.950438,True
1,Camille,Teemo,Tristana,Yuumi,Viego,Singed,Yone,Jhin,Pyke,Taliyah,52.311781,47.116516,51.078886,49.761747,49.082066,50.407886,48.790814,49.916269,49.529812,50.834221,52.248311,43.497758,51.250478,49.827638,49.100754,50.867721,48.712121,49.941680,49.052237,51.181453,2.095402,1.733938,2.467593,4.286643,2.414268,2.223274,1.901734,2.889686,2.268547,2.684889,2.088099,1.906325,2.510202,4.290919,2.595813,2.204869,1.981926,2.892598,2.214440,2.816094,False
2,Ornn,Zed,Jinx,Lulu,FiddleSticks,Shen,Sylas,Samira,Leona,XinZhao,49.782230,49.523810,50.973808,50.715869,51.464693,51.536406,50.200375,49.760483,50.387911,51.869731,49.851088,49.647516,50.998588,50.813931,51.669086,51.637952,50.421731,49.843517,50.450267,52.010744,2.439610,2.277548,2.504318,3.207114,2.796019,2.912215,2.127615,2.293444,2.442175,2.538234,2.436134,2.282057,2.505360,3.213491,2.843977,2.933312,2.200415,2.301646,2.443675,2.574962,True
3,Jayce,Sylas,Ezreal,Yuumi,Kayn,Volibear,AurelionSol,Caitlyn,Shaco,Viego,47.357867,50.200375,49.461616,49.761747,50.689906,48.582135,52.156177,48.906789,50.079104,49.082066,47.281568,50.421731,49.573524,49.827638,50.805617,48.416390,52.816020,48.984722,47.442958,49.100754,1.812713,2.127615,2.577849,4.286643,2.611583,2.351976,2.690024,2.326337,2.705159,2.414268,1.753307,2.200415,2.586941,4.290919,2.627467,1.927295,2.735680,2.331698,2.396533,2.595813,False
4,Camille,Kayle,Jhin,Shaco,Rengar,Sett,Leblanc,Tristana,Yuumi,Ekko,52.311781,50.966317,49.916269,50.079104,49.212720,49.702128,50.546767,51.078886,49.761747,50.805760,52.248311,50.744735,49.941680,47.442958,47.426299,50.862777,50.631533,51.250478,49.827638,50.300100,2.095402,2.025921,2.889686,2.705159,2.278364,1.905122,2.966847,2.467593,4.286643,2.651710,2.088099,2.303158,2.892598,2.396533,2.393806,1.855354,2.979701,2.510202,4.290919,2.853320,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126257,Mordekaiser,Sylas,Ezreal,Twitch,Shaco,Shen,Yone,Jhin,Seraphine,Evelynn,48.677020,50.200375,49.461616,49.673747,50.079104,51.536406,48.790814,49.916269,50.272336,49.061958,48.887390,50.421731,49.573524,40.240964,50.741840,51.637952,48.712121,49.941680,49.627200,49.095238,1.752603,2.127615,2.577849,2.381066,2.705159,2.912215,1.901734,2.889686,2.792648,2.940007,1.697270,2.200415,2.586941,1.889516,2.792898,2.933312,1.981926,2.892598,2.676447,2.950438,True
126258,Camille,Velkoz,Tristana,Rakan,Graves,Shen,Yone,Draven,Karma,Nidalee,52.311781,50.338219,51.078886,50.733138,49.068417,51.536406,48.790814,50.489022,48.568846,49.970369,52.248311,51.813153,51.250478,50.536661,48.596939,51.637952,48.712121,50.537079,48.794383,50.079124,2.095402,2.467497,2.467593,3.094978,2.468806,2.912215,

## Handling Categorical Variables
The champion values are converted into numerical values for the model to understand

In [7]:
# creating a mpping of champion names to numerical values e.g. Shen = 1 
unique_champs = dataset.CHAMPION.unique().tolist()
champs_map = {}
for i, champ in enumerate(unique_champs):
    champs_map[champ] = i

In [8]:
#Converting the categorical values into numerical by using the mapping created earlier
for x in tf_df.columns.tolist()[:10]:
    tf_df[x] = tf_df[x].map(champs_map)

In [9]:
tf_df.head(3)

,t1_top,t1_middle,t1_bottom,t1_utility,t1_jungle,t2_top,t2_middle,t2_bottom,t2_utility,t2_jungle,t1_top_wr,t1_middle_wr,t1_bottom_wr,t1_utility_wr,t1_jungle_wr,t2_top_wr,t2_middle_wr,t2_bottom_wr,t2_utility_wr,t2_jungle_wr,t1_top_role_wr,t1_middle_role_wr,t1_bottom_role_wr,t1_utility_role_wr,t1_jungle_role_wr,t2_top_role_wr,t2_middle_role_wr,t2_bottom_role_wr,t2_utility_role_wr,t2_jungle_role_wr,t1_top_kda,t1_middle_kda,t1_bottom_kda,t1_utility_kda,t1_jungle_kda,t2_top_kda,t2_middle_kda,t2_bottom_kda,t2_utility_kda,t2_jungle_kda,t1_top_role_kda,t1_middle_role_kda,t1_bottom_role_kda,t1_utility_role_kda,t1_jungle_role_kda,t2_top_role_kda,t2_middle_role_kda,t2_bottom_role_kda,t2_utility_role_kda,t2_jungle_role_kda,t1_win
0,1,3,5,7,9,0,2,4,6,8,52.233364,51.429580,49.760483,48.664122,50.689906,51.263264,48.790814,48.928255,49.915314,49.061958,52.334540,51.168224,49.843517,45.492289,50.805617,56.422018,48.712121,48.938667,50.100690,49.095238,1.827452,2.412114,2.293444,2.151817,2.611583,2.582077,1.901734,2.531856,2.354769,2.940007,1.827214,2.455387,2.301646,1.936946,2.627467,2.400353,1.981926,2.534069,2.355415,2.950438,True
1,11,12,14,16,18,10,2,13,15,17,52.311781,47.116516,51.078886,49.761747,49.082066,50.407886,48.790814,49.916269,49.529812,50.834221,52.248311,43.497758,51.250478,49.827638,49.100754,50.867721,48.712121,49.941680,49.052237,51.181453,2.095402,1.733938,2.467593,4.286643,2.414268,2.223274,1.901734,2.889686,2.268547,2.684889,2.088099,1.906325,2.510202,4.290919,2.595813,2.204869,1.981926,2.892598,2.214440,2.816094,False
2,142,40,19,21,23,94,26,5,20,22,49.782230,49.523810,50.973808,50.715869,51.464693,51.536406,50.200375,49.760483,50.387911,51.869731,49.851088,49.647516,50.998588,50.813931,51.669086,51.637952,50.421731,49.843517,50.450267,52.010744,2.439610,2.277548,2.504318,3.207114,2.796019,2.912215,2.127615,2.293444,2.442175,2.538234,2.436134,2.282057,2.505360,3.213491,2.843977,2.933312,2.200415,2.301646,2.443675,2.574962,True


## Prepocessing 

In [10]:
# dropping na values if any 
df  = tf_df[~tf_df.isna().any(axis=1)]
# converting dependent variable into 0 or 1 
df['t1_win'] = df['t1_win'].astype('int')
#converting numerical columns as int
df[df.columns[:10]] = df[df.columns[:10]].astype('int')
df[df.columns[-1]] = df[df.columns[-1]].astype('int')

In [11]:
tf_df.head(3)

,t1_top,t1_middle,t1_bottom,t1_utility,t1_jungle,t2_top,t2_middle,t2_bottom,t2_utility,t2_jungle,t1_top_wr,t1_middle_wr,t1_bottom_wr,t1_utility_wr,t1_jungle_wr,t2_top_wr,t2_middle_wr,t2_bottom_wr,t2_utility_wr,t2_jungle_wr,t1_top_role_wr,t1_middle_role_wr,t1_bottom_role_wr,t1_utility_role_wr,t1_jungle_role_wr,t2_top_role_wr,t2_middle_role_wr,t2_bottom_role_wr,t2_utility_role_wr,t2_jungle_role_wr,t1_top_kda,t1_middle_kda,t1_bottom_kda,t1_utility_kda,t1_jungle_kda,t2_top_kda,t2_middle_kda,t2_bottom_kda,t2_utility_kda,t2_jungle_kda,t1_top_role_kda,t1_middle_role_kda,t1_bottom_role_kda,t1_utility_role_kda,t1_jungle_role_kda,t2_top_role_kda,t2_middle_role_kda,t2_bottom_role_kda,t2_utility_role_kda,t2_jungle_role_kda,t1_win
0,1,3,5,7,9,0,2,4,6,8,52.233364,51.429580,49.760483,48.664122,50.689906,51.263264,48.790814,48.928255,49.915314,49.061958,52.334540,51.168224,49.843517,45.492289,50.805617,56.422018,48.712121,48.938667,50.100690,49.095238,1.827452,2.412114,2.293444,2.151817,2.611583,2.582077,1.901734,2.531856,2.354769,2.940007,1.827214,2.455387,2.301646,1.936946,2.627467,2.400353,1.981926,2.534069,2.355415,2.950438,True
1,11,12,14,16,18,10,2,13,15,17,52.311781,47.116516,51.078886,49.761747,49.082066,50.407886,48.790814,49.916269,49.529812,50.834221,52.248311,43.497758,51.250478,49.827638,49.100754,50.867721,48.712121,49.941680,49.052237,51.181453,2.095402,1.733938,2.467593,4.286643,2.414268,2.223274,1.901734,2.889686,2.268547,2.684889,2.088099,1.906325,2.510202,4.290919,2.595813,2.204869,1.981926,2.892598,2.214440,2.816094,False
2,142,40,19,21,23,94,26,5,20,22,49.782230,49.523810,50.973808,50.715869,51.464693,51.536406,50.200375,49.760483,50.387911,51.869731,49.851088,49.647516,50.998588,50.813931,51.669086,51.637952,50.421731,49.843517,50.450267,52.010744,2.439610,2.277548,2.504318,3.207114,2.796019,2.912215,2.127615,2.293444,2.442175,2.538234,2.436134,2.282057,2.505360,3.213491,2.843977,2.933312,2.200415,2.301646,2.443675,2.574962,True


In [12]:
# Scaling the feature values to have have uniform scales
scaler = preprocessing.StandardScaler()
scaled_cols = pd.DataFrame(scaler.fit_transform(df[df.columns[10:-1]]), index = df.index, columns = df.columns[10:-1])
df[scaled_cols.columns] = scaled_cols

In [13]:
df.head()

,t1_top,t1_middle,t1_bottom,t1_utility,t1_jungle,t2_top,t2_middle,t2_bottom,t2_utility,t2_jungle,t1_top_wr,t1_middle_wr,t1_bottom_wr,t1_utility_wr,t1_jungle_wr,t2_top_wr,t2_middle_wr,t2_bottom_wr,t2_utility_wr,t2_jungle_wr,t1_top_role_wr,t1_middle_role_wr,t1_bottom_role_wr,t1_utility_role_wr,t1_jungle_role_wr,t2_top_role_wr,t2_middle_role_wr,t2_bottom_role_wr,t2_utility_role_wr,t2_jungle_role_wr,t1_top_kda,t1_middle_kda,t1_bottom_kda,t1_utility_kda,t1_jungle_kda,t2_top_kda,t2_middle_kda,t2_bottom_kda,t2_utility_kda,t2_jungle_kda,t1_top_role_kda,t1_middle_role_kda,t1_bottom_role_kda,t1_utility_role_kda,t1_jungle_role_kda,t2_top_role_kda,t2_middle_role_kda,t2_bottom_role_kda,t2_utility_role_kda,t2_jungle_role_kda,t1_win
0,1,3,5,7,9,0,2,4,6,8,1.442021,0.973155,-0.035042,-1.614669,0.485151,0.832438,-0.665461,-0.697962,-0.406163,-0.756531,1.110467,0.616318,-0.087152,-2.360488,0.430402,3.051171,-0.689562,-0.561027,0.045562,-0.506485,-0.916960,0.260825,-0.602560,-1.388538,-0.144415,1.423442,-1.340449,0.512982,-0.976641,0.905089,-0.605601,0.291169,-0.723997,-1.665318,-0.480758,1.375885,-1.164857,0.381613,-0.859781,0.557656,1
1,11,12,14,16,18,10,2,13,15,17,1.491076,-1.713224,1.011984,-0.558959,-0.739977,0.298255,-0.665461,0.086572,-0.776366,0.593622,1.069534,-3.471772,0.651139,-0.102579,-0.498544,0.413303,-0.689562,-0.035788,-0.493630,0.637244,-0.084525,-1.867405,0.208755,2.924178,-0.775153,0.309550,-1.340449,2.183292,-1.150120,0.090818,0.296859,-1.400275,0.265365,2.854389,-0.582578,0.698543,-1.164857,2.079896,-1.129159,0.126450,0
2,142,40,19,21,23,94,26,5,20,22,-0.091292,-0.213849,0.928535,0.358728,1.075517,1.003015,0.210927,-0.037130,0.047678,1.382499,-0.068447,-0.194166,0.518962,0.411095,0.900888,0.779104,0.217849,-0.087192,0.225340,1.091888,0.984819,-0.161465,0.379847,0.743343,0.445157,2.448345,-0.632317,-0.599898,-0.800781,-0.377270,1.500792,-0.242793,0.242398,0.785691,0.215693,3.222567,-0.492326,-0.719334,-0.691131,-0.647516,1
3,24,26,28,16,9,25,27,29,30,18,-1.607858,0.207548,-0.272392,-0.558959,0.485151,-0.841925,1.426938,-0.715007,-0.248873,-0.741212,-1.288218,0.218463,-0.228829,-0.102579,0.430402,-0.750893,1.488668,-0.536909,-1.321240,-0.503461,-0.962750,-0.631980,0.722409,2.924178,-0.144415,0.709099,1.130817,-0.446358,-0.271656,-0.772937,-0.861262,-0.494299,0.629409,2.854389,-0.480758,-0.263241,1.155273,-0.576981,-0.781210,-0.580588,0
4,11,32,13,30,130,31,33,14,16,59,1.491076,0.684613,0.088677,-0.253720,-0.640423,-0.142491,0.426295,1.009749,-0.553635,0.571940,1.069534,0.390614,-0.035642,-1.344553,-1.410921,0.410955,0.329206,0.649579,-0.094862,0.154058,-0.084525,-0.951113,2.175176,-0.270691,-1.209587,-0.678143,1.998647,0.213010,2.910291,-0.015082,0.296859,-0.177787,2.079404,-0.782895,-1.232375,-0.512512,1.906396,0.268556,2.838638,0.245935,1


In [14]:
# splitting the dataset into test and train for modelling [80% Train & 20% Test]
# Saving independent values in X
X = df[df.columns[:-1]]
# Saving dependent values in Y
y = df[[df.columns[-1]]]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20)
print(X_train.shape)
print(X_test.shape)
print("train: ", y_train.value_counts())
print("test: ", y_test.value_counts())

(101009, 50)
(25253, 50)
train:  t1_win
1         50576
0         50433
dtype: int64
test:  t1_win
0         12699
1         12554
dtype: int64


## Modelling
We would be using XGBoost and Random Forest model to train the data to predict who would potentially win a match given the champion and the corresponding role the champion is played in. 

To better improve the prediction capability we would be trying built a champion model to outperform the XGBoost and Random Forest model 

###  Training the dataset on XGBoost Classifier 

Note: will be using this accuracy as baseline

In [27]:
# the model is trained on XGBoost algortihm 
clf_baseline = XGBClassifier()
clf_baseline.fit(X_train, y_train)
clf_baseline.score(X_test, y_test)

- The XGBoost classifer has an accuracy of 52% 

###  Training the dataset on Random Forest

In [20]:
clf = RandomForestClassifier(n_estimators=500, max_depth=10, n_jobs=-1)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)

In [26]:
print('Testing Accuracy - {0}'.format(clf.score(X_test, y_test)))
print('Training Accuracy - {0}'.format(clf.score(X_train, y_train)))

print('Precision {0}'.format(precision_score(y_test, y_pred )))
print('Recall {0}'.format(recall_score(y_test, y_pred )))

Testing Accuracy - 0.5470637152021542
Training Accuracy - 0.68226593669871
Precision 0.5446900528592024
Recall 0.5417396845626892


- The random forest algorithm performs slightly better than the XGBoost model 

### Training the dataset using Neural Networks 

In [ ]:
#importing necessary modules
from keras.layers import *
from keras.layers import Dense, Activation, Flatten
from keras.layers import Bidirectional, CuDNNLSTM,LSTM
from keras.regularizers import *
from keras import regularizers
from keras.constraints import max_norm 
from keras.models import Model, Sequential

from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras import activations, initializers, regularizers, constraints
from keras.utils.conv_utils import conv_output_length
from keras.regularizers import l2
from keras.constraints import maxnorm
# from keras.utils import multi_gpu_model

In [ ]:
#Neural Network Architecture 
x1 = Input(1170)
x2 = Dropout(0.4)(x1)
x3 = Dense(50, activation = 'relu')(x2)
x4 = Dense(50, activation = 'relu')(x3)
x5= Dense(32, activation = 'relu')(x4)
x6= Dense(5, activation = 'relu')(x5)
xfinal = Dense(1, activation = 'sigmoid')(x5)
model = Model(x1,xfinal)
model.compile(loss = 'binary_crossentropy', optimizer = 'nadam', metrics = ['acc'])
model.fit(X_train,y_train, epochs=10, batch_size = 64, verbose = 1, validation_data=(X_test,y_test))

- The accuracy of the Neural Network model is at 54% 